In [1]:
import json
import os
import random
import numpy as np
from tqdm import tqdm

import torch
from tqdm import tqdm
import random
import seaborn as sns
from pylab import rcParams

from SkeletonData.data import *

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

### Add Skeleton of selected videos

In [2]:
#action_npz_file = "../data/skeleton_k10_v7_ movements.npz"
#action_arr = np.load(action_npz_file)

In [3]:
#for cls,arr,vid_n in zip(action_arr["arr_0"],action_arr["arr_1"],action_arr["arr_2"]):
#    os.makedirs(os.path.join(npz_files,f"{cls}"),exist_ok=True)
#    np.savez(os.path.join(npz_files,f"{cls}/{vid_n}.npz"),coords)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
mapping_l = [
        [15, 13], [13, 11], [11, 5],
        [12, 14], [14, 16], [12, 6],
        [3, 1],[1, 2],[1, 0],[0, 2],[2,4],
        [9, 7], [7,5], [5, 6],
        [6, 8], [8, 10],
        ]
#mapping_l = []

## Sequence Data

In [6]:
root_dir = "D:\\FYP\\HAR-ZSL-XAI\\AETraining"
main_dir = "D:\\FYP\\HAR-ZSL-XAI"
data_dir = os.path.join(main_dir,"pyskl/HRNetSkeletons/nipun_video_dataset/PAMAP2_K10_V1/combined.pkl")
refined_data = os.path.join(main_dir,"data","sequence_data","random_50f")
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 1-train_ratio - val_ratio
batch_size = 32

In [7]:
class_names = os.listdir(os.path.join(main_dir,"data/nipun_video_dataset/PAMAP2_K10_V1"))

In [8]:
from SkeletonData.array_segment import *
from SkeletonData.visualize import *

In [9]:
pkl_reader = SkeletonPickleReader(file_path="D:/FYP/HAR-ZSL-XAI/pyskl/HRNetSkeletons/nipun_video_dataset/PAMAP2_K10_V1/combined.pkl")
#pkl_reader = SkeletonPickleReader(file_path="E:\\FYP_Data\\NTU120\\ntu120_hrnet.pkl")
anno = pkl_reader.generate(split=False)

In [10]:
files = FileDictIterator(anno)

In [14]:
from functools import partial
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

### Visualize

In [16]:
def visualize_after_normalize(__f):
    kp = __f["keypoint"][0]
    kp[:,:,0] = kp[:,:,0]/__f["original_shape"][1]
    kp[:,:,1] = kp[:,:,1]/__f["original_shape"][0]

    gen_video(kp,f"temp_files/{__f['frame_dir']}.mp4",__f["original_shape"][0],__f["original_shape"][1],mapping_list=mapping_l)

    return f"temp_files/{__f['frame_dir']}.mp4"

In [17]:
os.makedirs("temp_files",exist_ok=True)

with ThreadPoolExecutor() as executor:
    file_loc = list(tqdm(executor.map(visualize_after_normalize,files), total=len(files),desc="Saved Videos:"))

Saved Videos::   0%|          | 52/113945 [00:25<15:49:06,  2.00it/s]


KeyboardInterrupt: 

### Break Into Segments

In [11]:
 files = iter(files)

In [12]:
id2shapes = {}
for k,v in anno.items():
    id2shapes[k] = v["original_shape"]


id2shapes

{'as10': (1080, 1920),
 'as12': (1080, 1920),
 'as13': (1080, 1920),
 'as14': (1080, 1920),
 'as15': (1080, 1920),
 'as16': (1080, 1920),
 'as18': (1080, 1920),
 'as7': (1080, 1920),
 'as8': (1080, 1920),
 'as9': (1080, 1920),
 'cd11': (1080, 1920),
 'cd13': (1080, 1920),
 'cd14': (1080, 1920),
 'cd16': (1080, 1920),
 'cd18': (1080, 1920),
 'cd19': (1080, 1920),
 'cd2': (1080, 1920),
 'cd5': (1080, 1920),
 'cd6': (1080, 1920),
 'cd9': (1080, 1920),
 'c10': (1080, 1920),
 'c11': (1080, 1920),
 'c13': (1080, 1920),
 'c17': (1080, 1920),
 'c18': (1080, 1920),
 'c19': (1080, 1920),
 'c2': (1080, 1920),
 'c23': (1080, 1920),
 'c24': (1080, 1920),
 'c6': (1080, 1920),
 'c14': (1080, 1920),
 'c16': (1080, 1920),
 'c20': (1080, 1920),
 'c4': (1080, 1920),
 'c9': (1080, 1920),
 'ds1': (1080, 1920),
 'ds10': (1080, 1920),
 'ds12': (1080, 1920),
 'ds13': (1080, 1920),
 'ds18': (1080, 1920),
 'ds20': (1080, 1920),
 'ds22': (1080, 1920),
 'ds5': (1080, 1920),
 'ds7': (1080, 1920),
 'ds9': (1080, 19

In [15]:
os.makedirs(refined_data,exist_ok=True)
#for each_file in tqdm(files,desc="Files Used",total=len(files)):
with ThreadPoolExecutor() as executor:
    file_loc = list(tqdm(executor.map(partial(split_array_main,id2shapes,refined_data),files), total=len(files),desc="Processed Files:"))

Processed Files:: 100%|██████████| 167/167 [00:00<?, ?it/s]
